In [1]:
# %matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
from astropy import units as u
import tqdm 
import matplotlib.animation as anim
from astropy.visualization import astropy_mpl_style, quantity_support
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation

### 17th Feb:

**Conditional about observations:**

The user gives the latitude and the functions returns if the phenomenon can be see it or not. This using the conditional "altitude > 0 degrees" but with the difference of taking into account the time of observation on the altitude-Azimuth coordinate system; this is the equivalent ofr the conditional of "dec > 90-latitude".   

Alert system's input format: RA, Dec (hmsdms); Latitude, longitude (DMS); militar hour (hms) using J200 
(Reference: https://ztf.uw.edu/alerts/public/)

**About the sing on the coordinates**
 A positive value for North and East, a negative value for South and West. (Reference: https://tnp.uservoice.com/knowledgebase/articles/172110-latitude-longitude-formats-and-conversion)

### 21th Feb

Now we need to expand the function for a time slot, establish by the user. With this condition, is necesary to report if its observable. An idea is refreshing the dataframe with a nonsmall step, such we can determine if the function is returning what we want. 

Change the scale of time to UTC and keep the format iso for maintain the international format.

Actually, the function Time of astropy simplifies the code because allows sum seconds, hour and minutes in a line.
(Reference: https://docs.astropy.org/en/stable/api/astropy.time.Time.html#astropy.time.Time.FORMATS)

### 24 Feb - 3 Mar
This week have three objectives: 

1) Implement a preference condition, just show the observables and organized with preference of higher altitude. The preference it gonna be a color code:

    Priority: Extraordinary: Green;  High: Orange; Medium: Blue; Low: Pink

2) Try to simulate for see in the time slot only the observables with the observer as the center. First we need to get the coordinates of the observables in the altaz format.

3) Change the enter for a dataframe with the transients information.

In [2]:
pdt = pd.DataFrame([])
pdt['Time scale'] = ['1 hour','30 min','10 min','30 seg','10 seg','5 seg']
pdt['Time response'] = ['0.2 s','0.4 s','1.6 s','21.9 s','1m 1.6 s','2m 22.3s']
pdt

,Time scale,Time response
0,1 hour,0.2 s
1,30 min,0.4 s
2,10 min,1.6 s
3,30 seg,21.9 s
4,10 seg,1m 1.6 s
5,5 seg,2m 22.3s


In [3]:
#Playtime data

#Tuple in order of (latitude,longitude)
observer = '4-35-56-N' ,'74-04-51-W'  

#Data Frame with information of the objects, we are working with the three limit cases: Is observable, is not and is near to the horizon 
alert = pd.DataFrame ( {'Name': ['Aldebaran', 'Antares'],
                        'RA': ['04h35m55.64s','16h29m24.17s'],
                        'DEC':['+16d30m27.2s','-26d25m53.56s'] } ) 
 
#Slot of time and date
date_i = '2025-02-17 19:00:00'
date_f = '2025-02-18 07:00:00'

#Time scale of the observation, the minimum is 5 seconds
timescale = ['m', 30]

priority = ['Extraordinary','High','Medium','Low']

#Convert the DMS format to degrees
def ConvertLaLo(observer):
    convert = []
    for each in observer:
        ob_ = each.split('-')

        #DMS format
        dms = float(ob_[0]) + (float(ob_[1])/60) + (float(ob_[2])/3600)

        #Change the sign if its on west or sourth
        if ob_[3] == 'W' or ob_[3] == 'S':
            convert.append(dms*(-1))
        else:
            convert.append(dms)

    return convert

alert

,Name,RA,DEC
0,Aldebaran,04h35m55.64s,+16d30m27.2s
1,Antares,16h29m24.17s,-26d25m53.56s


In [4]:
# #Move the object time in astropy
# def MoveTime(time_actual,scale,sc):
#     if scale == 's':
#         t = time_actual + sc*u.second 
#     if scale == 'm':
#         t = time_actual + sc*u.minute
#     if scale == 'h':
#         t = time_actual + sc*u.hour

#     return t

In [5]:
# #With time slot and optimized

# def Observations(observer, alert, Date_i, Date_f,time_scale):

#     #Definition of different variables:
#     actual_time = Time(Date_i,format = 'iso', scale='utc')
#     final_time = Time(Date_f,format = 'iso', scale='utc')

#     lat_conv , lon_conv = ConvertLaLo(observer)
#     observer = EarthLocation(lat=lat_conv*u.deg, lon=lon_conv*u.deg)

#     #Lists of data for post-analysis
#     Big_Data = []

#     #Establish the time slot
#     while actual_time <= final_time:

#         #For each time slot we need the data of the conditional about observations
        
#         #Establish the coordinates with the correction from geocentric calculations of astropy
#         celestial_coord = SkyCoord(ra=alert['RA'], dec=alert['DEC']) #Mantain the degrees units

#         # Calculate the coordenates AltAz for the time and observer and transform the J2000 coordinate system for altaz
#         altaz_coord = celestial_coord.transform_to( AltAz(obstime=actual_time, location=observer) ) 
    
#         # Determinate if its observable (altitude > 0 degrees) in the altaz coordinate system, return a boolean.
#         state  = altaz_coord.alt > 0*u.deg

#         #Put the things together
#         alert['Observable'] = state
#         alert['Az'] = altaz_coord.az.deg
#         alert['Alt'] = altaz_coord.alt.deg
        
#         #Put the conditionals of a time slot on a dataframe in the format for better reading
#         Data = alert.copy()

#         #Eliminate the observations who the state is false
#         # Data = Data[Data['Observable'] != False]

#         #With the DataFrame we can organice by the higher altitude
#         Data = (Data.sort_values(by='Alt', ascending=False, na_position='first')).reset_index(drop=True)

#         #Establish the priority by a percent of the list. Because the list is organized by ascending altitude
#         # p = [priority[0]]*int((98/100)*len(Data['Name'])) + [priority[1]]*int((70/100)*len(Data['Name'])) + [priority[2]]*int((40/100)*len(Data['Name'])) + [priority[3]]*int((20/100)*len(Data['Name']))

#         # Data['Priority'] = p
        
#         #Put the dataframe in a list and the values of time
#         Big_Data.append(Data) 

#         #Actualize the time depending of the time scale
#         actual_time = MoveTime(actual_time,time_scale[0],time_scale[1])

#     return Big_Data

# Data = Observations(observer, alert, date_i, date_f,timescale)
# Data[0]

## 6- 9 Mar

In [6]:
#Made the array of time
def MoveTime(Date_i,Date_f,t_scale):

    scale = t_scale[0]
    sc = t_scale[1]

    a = Time( Date_i,format = 'iso', scale='utc').datetime.hour-24
    b = Time(Date_f,format = 'iso', scale='utc').datetime.hour

    time_midnight = Time( Time(Date_i,format = 'iso', scale='utc').iso.split()[0] + ' 00:00:00' ,format = 'iso', scale='utc')

    if scale == 's':
        len = int((b-a)/ (sc/3600))
        t_ = np.linspace(a , b , len)*u.hour 
        t = time_midnight + t_
    if scale == 'm':
        len = int((b-a)/ (sc/60))
        t_ = np.linspace(a , b , len)*u.hour
        t = time_midnight + t_
    if scale == 'h':
        len = int((b-a)/ sc)
        t_ = np.linspace(a , b , len)*u.hour
        t = time_midnight + t_

    return t

def MovTime(Date_i,Date_f,t_scale):

    scale = t_scale[0]
    sc = t_scale[1]

    a = Time( Date_i,format = 'iso', scale='utc').datetime.hour-24
    b = Time(Date_f,format = 'iso', scale='utc').datetime.hour

    if scale == 's':
        len = int((b-a)/ (sc/3600))
        t_ = np.linspace(a , b , len)*u.hour 
   
    if scale == 'm':
        len = int((b-a)/ (sc/60))
        t_ = np.linspace(a , b , len)*u.hour
    
    if scale == 'h':
        len = int((b-a)/ sc)
        t_ = np.linspace(a , b , len)*u.hour
    

    return t_

In [32]:
#With time slot and optimized

def Observations(observer, alert, Date_i, Date_f,time_scale):

    #Definition of different variables:
    time = MoveTime(Date_i,Date_f,time_scale) #return a array

    lat_conv , lon_conv = ConvertLaLo(observer)
    observer = EarthLocation(lat=lat_conv*u.deg, lon=lon_conv*u.deg)

    #Lists of data for post-analysis
    Big_Data = []

    #Establish the time slot
    for each_time in time:

        #For each time slot we need the data of the conditional about observations
        
        #Establish the coordinates with the correction from geocentric calculations of astropy
        celestial_coord = SkyCoord(ra=alert['RA'], dec=alert['DEC']) #Mantain the degrees units

        # Calculate the coordenates AltAz for the time and observer and transform the J2000 coordinate system for altaz
        altaz_coord = celestial_coord.transform_to( AltAz(obstime=each_time, location=observer) ) 
    
        # Determinate if its observable and a good observable (altitude > 33 degrees) in the altaz coordinate system, return a boolean.
        state  = altaz_coord.alt > 0*u.deg

        #Put the things together
        alert['Observable'] = state
        alert['Az'] = altaz_coord.az.deg
        alert['Alt'] = altaz_coord.alt.deg
        
        #Put the conditionals of a time slot on a dataframe in the format for better reading
        Data = alert.copy()

        #With the DataFrame we can organice by the higher altitude
        Data = (Data.sort_values(by='Alt', ascending=False, na_position='first'))

        #Establish the priority by a percent of the list. Because the list is organized by ascending altitude
        p = [priority[0]]*int((98/100)*len(Data['Name'])) + [priority[1]]*int((70/100)*len(Data['Name'])) + [priority[2]]*int((40/100)*len(Data['Name'])) + [priority[3]]*int((20/100)*len(Data['Name']))

        Data['Priority'] = p

        #Eliminate the observations who the state is false
        Data = Data[Data['Observable'] != False]
        
        #Put the dataframe in a list and the values of time
        Big_Data.append(Data) 

    return Big_Data

Data = Observations(observer, alert, date_i, date_f,timescale)
time = MovTime(date_i,date_f,timescale)


In [35]:
#Other functions

#Take the data from the analysis to graphic
def EstablishCoord(Data):
    ALT = []
    AZ = []
    
    for each in Data:
        ALT.append(each['Alt'])
        AZ.append(each['Az'])
    return ALT,AZ

EstablishCoord(Data)

([0    19.118499
  Name: Alt, dtype: float64,
  0    26.635441
  Name: Alt, dtype: float64,
  0    34.134142
  Name: Alt, dtype: float64,
  0    41.595737
  Name: Alt, dtype: float64,
  0    48.990652
  Name: Alt, dtype: float64,
  0    56.266417
  Name: Alt, dtype: float64,
  0    63.317245
  Name: Alt, dtype: float64,
  0    69.897123
  Name: Alt, dtype: float64,
  0    75.349461
  Name: Alt, dtype: float64,
  0    78.01523
  Name: Alt, dtype: float64,
  0    76.173873
  Name: Alt, dtype: float64,
  0    71.100573
  Name: Alt, dtype: float64,
  0    64.670108
  Name: Alt, dtype: float64,
  0    57.685756
  Name: Alt, dtype: float64,
  0    50.443512
  Name: Alt, dtype: float64,
  0    43.066942
  Name: Alt, dtype: float64,
  0    35.6157
  Name: Alt, dtype: float64,
  0    28.122595
  Name: Alt, dtype: float64,
  0    20.608045
  Name: Alt, dtype: float64,
  0    13.086507
  Name: Alt, dtype: float64,
  0    5.569777
  Name: Alt, dtype: float64,
  1    2.408178
  Name: Alt, dtype: fl

### Graphic sketch

In [9]:
# #Other functions

# #Take the data from the analysis to graphic
# def EstablishCoord(Data,i):
#     colors=['limegreen','darkorange','midnightblue','hotpink']
#     ALT =[]
#     AZ = []
#     preference = []
    
#     for each in Data:
#         ALT.append(each['Alt'][i])
#         AZ.append(each['Az'][i])

#         if each['Priority'][i] == 'Extraordinary':
#             preference.append(colors[0])
        
#         if each['Priority'][i] == 'High':
#             preference.append(colors[1])

#         if each['Priority'][i] == 'Medium':
#             preference.append(colors[2])

#         if each['Priority'][i] == 'Low':
#             preference.append(colors[3])

#     return ALT,AZ,preference


In [10]:
# plt.style.use(astropy_mpl_style)
# quantity_support()

# for i in range(0,2):
#     ALT,AZ,priority = EstablishCoord(Data,i)
#     midnight = np.linspace(-2, 20, len(ALT)) 
#     plt.scatter(midnight, ALT,color=priority)

# plt.show()

#### 3D plot

In [11]:
#Other functions

# #Take the data from the analysis to graphic
# def EstablishCoord(Data,i):
#     colors=['limegreen','darkorange','midnightblue','hotpink']
#     ALT =[]
#     AZ = []
#     X= []
#     Y= []
#     Z= []
#     preference = []
    
#     for each in Data:
#         ALT.append(each['Alt'][i])
#         AZ.append(each['Az'][i])
#         X.append(each['Altaz'][i].cartesian.x)
#         Y.append(each['Altaz'][i].cartesian.y)
#         Z.append(each['Altaz'][i].cartesian.z)
#         if each['Priority'][i] == 'Extraordinary':
#             preference.append(colors[0])
        
#         if each['Priority'][i] == 'High':
#             preference.append(colors[1])

#         if each['Priority'][i] == 'Medium':
#             preference.append(colors[2])

#         if each['Priority'][i] == 'Low':
#             preference.append(colors[3])

#     return ALT,AZ,X,Y,Z,preference

# #Pass from spherical coordinates to cartesian
# def TransformSphere(theta, phi, center,radius):
#     x = center[0] + radius * np.sin(phi) * np.cos(theta)
#     y = center[1] + radius * np.sin(phi) * np.sin(theta)
#     z = center[2] + radius * np.cos(phi)
#     return x, y, z

#Create a figure 
# fig1 = plt.figure(figsize=(6,4))  
# ax1 = fig1.add_subplot(projection='hammer') 
# #make the plot black
# fig1.patch.set_facecolor('k') 
# ax1.set_facecolor('gray')

# #Stablish the interface
# plt.style.use(astropy_mpl_style)
# quantity_support()

# # Plot the data of observables
# for i in range(0,int(len(Data[0]['Name']))):
#     alt,az,x,y,z,c = EstablishCoord(Data,i)

#     ax1.scatter(np.radians(az),np.radians(alt),color=c)

# #Create a figure 
# fig3d = plt.figure(figsize=(6,6))  
# ax3d = fig3d.add_subplot(projection='3d') 
# #make the plot black
# fig3d.patch.set_facecolor('k') 
# ax3d.set_facecolor('k')

# #Sphere parameters, for a projection we take the radius like one and the observer is on the center
# center = (0, 0, 0)
# radius = 1
# phi = np.linspace(0, np.pi/2, 100)
# theta = np.linspace(0, 2*np.pi, 100)

# #Create a mesh grid for the sphere and plotting the "sky"
# Phi, Theta = np.meshgrid(phi, theta)
# X , Y, Z = TransformSphere(Theta, Phi, center,radius)
# ax3d.plot_surface(X , Y, Z, cmap='YlGnBu', alpha=0.3) #Alpha is for oppacity

# #The center
# ax3d.scatter(center[0],center[1],center[2],color='r')

# # Plot the data of observables
# #First plot is for the astropy plot and second the projection in the hammer plot
# for i in range(0,len(Data[0]['Name'])):
#     alt,az,x,y,z,c = EstablishCoord(Data,i)
#     ax3d.scatter(x,y,z,marker='*',color=c)
